<a href="https://colab.research.google.com/github/spirosChv/smartNetsWorkshop/blob/main/brian2/Brian2_basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Main topics
* Understanding Brian's units system
* Describing a simple neuronal model
* Core Brian classes: `NeuronGroup`, `Synapses`, `Monitors`
* Running and recording simulations
* Output visualization


NOTE: Some examples were adapted by the official Brian 2 tutorials. To access the original tutorials as well as Brian's complete documentation please visit the [Brian 2 website](https://brian2.readthedocs.io/en/stable/index.html)

Author: Michalis Pagkalos, [Poirazi Lab](https://dendrites.gr/?page_id=265)

## Brian 2 installation

In [ ]:
!pip install brian2 --quiet # Same in all Linux systems

## Import useful stuff
**Important:** Our world would be a better place if `*` imports were not a thing. It might seem tempting to import everything from a given package at the beginning of your code, but in fact, it's the source of all evil. If you ever see any code of mine with \* star imports, please contact the police and let them know that: a) I have been kidnapped, b) Someone has stolen my identity, c) My cat needs feeding (no more than 80 gr / day,  he is fat). Thank you.

In [ ]:
import brian2 as b
from brian2.units import volt, amp, namp, ohm, nA, ms, mV, pF, nS, pA

b.prefs.codegen.target = 'numpy' # pretend this line doesn't exist for now

## Units system

Brian has a system for using quantities with physical dimensions:

In [ ]:
20*volt

All of the basic SI units can be used (volt, amp, etc.) along with all the standard prefixes (m=milli, p=pico, etc.), as well as a few special abbreviations like `mV` for millivolt, `pF` for picofarad, etc.

In [ ]:
1000*amp

In [ ]:
1000*namp

In [ ]:
1e6*volt

Also note that combinations of units with work as expected:

In [ ]:
10*amp * 5*ohm

And if you try to do something wrong like adding amps and volts, what happens?

In [ ]:
5*amp + 10*volt

## A simple model

Let's start by defining a simple neuron model. In Brian, all models are defined by systems of differential equations. Here's a simple example of what that looks like:

In [ ]:
eqs = '''dv/dt = (1-v)/tau : 1'''

In Python, the notation `'''` is used to begin and end a multi-line string. So the equations are just a string with one line per equation. The equations are formatted with standard mathematical notation, with one addition. At the end of a line you write `: unit` where `unit` is the SI unit of that variable.
Note that this is not the unit of the two sides of the equation (which would be `1/second`), but the unit of the *variable* defined by the equation, i.e. in this case $v$.

Now let's use this definition to create a neuron and run our first simulation.

In [ ]:
b.start_scope()

# model equations:
eqs = '''dv/dt = (1-v)/tau : 1'''

# make a neurongroup:
G = b.NeuronGroup(1, eqs, method='euler')

# record how the v variable evolved during the simulation:
M = b.StateMonitor(G, 'v', record=0)  # record=True to record from all neurons

# tell Brian to run simulation
b.run(100*ms)

Oops something is missing. Let's fix it.

In [ ]:
b.start_scope()

# model equations and params :
eqs = '''dv/dt = (1-v)/tau : 1'''
tau = 10*ms # membrane time constant

# make a neurongroup:
G = b.NeuronGroup(1, eqs, method='euler')

# record how the v variable evolved during the simulation:
M = b.StateMonitor(G, 'v', record=0)  # record=True to record from all neurons

# tell Brian to run simulation
b.run(100*ms)

First off, ignore that `start_scope()` at the top of the cell. You'll see that in each cell in this tutorial where we run a simulation. All it does is make sure that any Brian objects created before the function is called aren't included in the next run of the simulation.

In Brian, you only create groups of neurons, using the class `NeuronGroup`. The first two arguments when you create one of these objects are the number of neurons (in this case, 1) and the defining differential equations.

The object ``StateMonitor`` object is used to record the values of a neuron variable while the simulation runs. The first two arguments are the group to record from, and the variable you want to record from. We also specify ``record=0``. This means that we record all values for neuron 0. We have to specify which neurons we want to record because in large simulations with many neurons it usually uses up too much RAM to record the values of all neurons.

In [ ]:
b.plot(M.t/ms, M.v[0])
b.xlabel('Time (ms)')
b.ylabel('v');

## Time for REAL fun
Now that you have general understanding of how Brian 2 works let's play a little and explore what else we can do.

### Example 1: Multiple neurons

In [ ]:
b.start_scope()

# make model and initialise v
eqs = '''dv/dt = (1-v)/tau : 1'''
tau = 10*ms 
G = b.NeuronGroup(2, eqs, method='euler')
G.v[0] = 0
G.v[1] = 3 

# set v monitors
M = b.StateMonitor(G, 'v', record=True)  # record=True to record from all neurons

# run simulation
b.run(100*ms)

# plot output
b.plot(M.t/ms, M.v[0], label='neuron 0')
b.plot(M.t/ms, M.v[1], label='neuron 1')
b.xlabel('Time (ms)')
b.ylabel('v')
b.legend();

### Example 2: Leaky IF

In [ ]:
b.start_scope()

C = 50 * pF # capacitance
gL = 5 * nS # leak conductance
EL = -60*mV # rest voltage
Vt = -40*mV # spike threshold
Vr = -50*mV # voltage reset after spike

eqs = '''
dv/dt = (gL*(EL-v) + I) / C  : volt
I : amp
'''

# create model
G = b.NeuronGroup(1, eqs, threshold='v>Vt', reset='v = Vr', method='euler')
G.v = EL # initialise rest voltage

# set v monitors
M = b.StateMonitor(G, 'v', record=0)

# first 20 ms -> no input
b.run(20*ms)
# next 100 ms -> 120 pA current injection
G.I = 120*pA
b.run(100*ms)
# next 50 ms -> no input to return to rest
G.I = 0*pA
b.run(50*ms)

# plot output
b.plot(M.t/ms, M.v[0]/mV)
b.xlabel('Time (ms)')
b.ylabel('Voltage (mV)');

### Example 3: Counting spikes

In [ ]:
b.start_scope()
b.seed(123)
G = b.NeuronGroup(50, eqs, threshold='v>Vt', reset='v = Vr', method='euler')
G.v = b.uniform(-60, -45, 50) * mV # initialise rest voltage

S = b.SpikeMonitor(G)

G.I = 120*pA
b.run(100*ms)

b.plot(S.t/ms, S.i, '.k')
b.xlabel('Time (ms)')
b.ylabel('Neuron index');

In [ ]:
S.t  # spiketimes

In [ ]:
S.i  # spike indices

In [ ]:
S.all_values()

### Example 4: Synapses

In [ ]:
b.start_scope()
G = b.NeuronGroup(2, eqs, threshold='v>Vt', reset='v = Vr', method='euler')
G.v = EL

S = b.Synapses(G, G, on_pre='v_post += 4*mV', delay=2*ms)
S.connect(i=0, j=1)

M = b.StateMonitor(G, 'v', record=True)
Spikes = b.SpikeMonitor(G)

G.I[0] = 120*pA
b.run(100*ms)

b.plot(M.t/ms, M.v[0]/mV, label='neuron 0')
b.plot(M.t/ms, M.v[1]/mV, label='neuron 1')
for t in Spikes.t:
    b.axvline(t/ms, ls='--', c='red', lw=1)
b.xlabel('Time (ms)')
b.ylabel('v')
b.legend();

### Example 5: AMPA / NMDA currents

In [ ]:
b.start_scope()
# equations and parameters
eqs = '''
dv/dt = (gL*(EL-v) + I + I_ampa) / C  : volt
I_ampa = g_ampa * (E_ampa-v) * s : amp
ds/dt = -s / tau_ampa : 1
I : amp
'''
E_ampa = 0*mV
g_ampa = 2*nS
tau_ampa = 2*ms

# create model
G = b.NeuronGroup(2, eqs, threshold='v>Vt', reset='v = Vr', method='euler')
G.v = EL # initialise rest voltage

# synaspses
S = b.Synapses(G, G, on_pre='s += 1', delay=2*ms)
S.connect(i=0, j=1)

# monitors
M = b.StateMonitor(G, 'v', record=True)
Spikes = b.SpikeMonitor(G)

# simulation
G.I[0] = 120*pA
b.run(100*ms)

# plotting
b.plot(M.t/ms, M.v[0]/mV, label='neuron 0')
b.plot(M.t/ms, M.v[1]/mV, label='neuron 1')
for t in Spikes.t:
    b.axvline(t/ms, ls='--', c='red', lw=1)
b.xlabel('Time (ms)')
b.ylabel('v')
b.legend();

In [ ]:
b.start_scope()
# equations and parameters
eqs = '''
dv/dt = (gL*(EL-v) + I + I_syn) / C  : volt
I_ampa = g_ampa * (E_ampa-v) * s_ampa : amp
ds_ampa/dt = -s_ampa / tau_ampa : 1

I_nmda = g_nmda * (E_nmda-v) * s_nmda / sigma : amp
ds_nmda/dt = -s_nmda / tau_nmda : 1
sigma = 1 + Mg * exp(-alpha*(v/mV+gamma)) / beta : 1

I_syn = I_ampa + I_nmda : amp
I : amp
'''

g_ampa = 0.3*nS
tau_ampa = 2*ms
E_ampa = 0*mV

g_nmda = 0.3*nS
tau_nmda = 50*ms
E_nmda = 0*mV
alpha = 0.062
beta = 3.57
gamma = 0
Mg = 1.0

# create model
G = b.NeuronGroup(1, eqs, method='euler')
G.v = -60*mV

# create input
I = b.SpikeGeneratorGroup(1, [0], [10*ms])

# synapses
S = b.Synapses(I, G, on_pre='s_ampa += 1; s_nmda += 1')
S.connect(i=0, j=0)

# monitors
M = b.StateMonitor(G, ['v', 'I_ampa', 'I_nmda'], record=True)

# simulation
b.run(200*ms)

# plotting
fig, (ax1, ax2) = b.subplots(1, 2, figsize=[10, 4])
ax1.plot(M.t/ms, M.v[0]/mV, label='neuron 0')
ax2.plot(M.t/ms, -M.I_ampa[0]/pA, c='black', label='I_ampa')
ax2.plot(M.t/ms, -M.I_nmda[0]/pA, c='red', label='I_nmda')
ax1.set_xlabel('Time (ms)')
ax1.set_ylabel('Voltage (mV)')
ax2.set_xlabel('Time (ms)')
ax2.set_ylabel('Current (pA)')
ax2.legend()
fig.tight_layout();

In [ ]:
b.start_scope()
EL = -20*mV
# create model
G = b.NeuronGroup(1, eqs, method='euler')
G.v = EL

# create input
I = b.SpikeGeneratorGroup(1, [0], [10*ms])

# synapses
S = b.Synapses(I, G, on_pre='s_ampa += 1; s_nmda += 1')
S.connect(i=0, j=0)

# monitors
M = b.StateMonitor(G, ['v', 'I_ampa', 'I_nmda'], record=True)

# simulation
b.run(200*ms)

# plotting
fig, (ax1, ax2) = b.subplots(1, 2, figsize=[10, 4])
ax1.plot(M.t/ms, M.v[0]/mV, label='neuron 0')
ax2.plot(M.t/ms, -M.I_ampa[0]/pA, c='black', label='I_ampa')
ax2.plot(M.t/ms, -M.I_nmda[0]/pA, c='red', label='I_nmda')
ax1.set_xlabel('Time (ms)')
ax1.set_ylabel('Voltage (mV)')
ax2.set_xlabel('Time (ms)')
ax2.set_ylabel('Current (pA)')
ax2.legend()
fig.tight_layout();